In [25]:
import pandas as pd
from sklearn import ensemble
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np

columnName = ['seed', 'lat', 'long'	, 'fg2pct', 'fg3pct', 'ftpct', 'blockpct', 'oppfg2pct', 'oppfg3pct', 'oppftpct',
              'oppblockpct', 'f3grate', 'oppf3grate', 'arate', 'opparate', 'stlrate', 'oppstlrate', 'tempo', 'adjtempo',
              'oe', 'adjoe', 'de', 'adjde', 'rpi_rating', 'pt_school_ncaa', 'pt_overall_ncaa', 'pt_school_s16', 'pt_overall_s16',
              'pt_school_ff', 'pt_overall_ff', 'pt_career_school_wins',	'pt_career_school_losses', 'pt_career_overall_losses',
              'team_season_wins', 'pt_team_season_losses', 'pt_coach_season_wins', 'pt_coach_season_losses']

df = pd.read_csv('NCAA_Tourney_2002-2016_1.csv')
features = df.iloc[:, :-4]
results = df.iloc[:, -4:]

df_team1 = df.iloc[:, 2:39]
df_team1.columns = columnName
df_team2 = df.iloc[:, 39:76]
df_team2.columns = columnName

df_team1.loc[:, 'expwin'] = df.loc[:, 'team1_adjoe'] ** 11.5 / (df.loc[:, 'team1_adjde'] ** 11.5
                                                            + df.loc[:, 'team1_adjoe'] ** 11.5)


df_team2.loc[:, 'expwin'] = df.loc[:, 'team2_adjoe'] ** 11.5 / (df.loc[:, 'team2_adjde'] ** 11.5
                                                            + df.loc[:, 'team2_adjoe'] ** 11.5)


df_team1.loc[:, 'log5'] = (df_team1.loc[:, 'expwin'] - df_team1.loc[:, 'expwin'] * df_team2.loc[:, 'expwin']) / \
            ((df_team1.loc[:, 'expwin'] + df_team2.loc[:, 'expwin'] - 2* df_team1.loc[:, 'expwin'] * df_team2.loc[:, 'expwin']))

df_team2.loc[:, 'log5'] = (df_team2.loc[:, 'expwin'] - df_team1.loc[:, 'expwin'] * df_team2.loc[:, 'expwin']) / \
            ((df_team1.loc[:, 'expwin'] + df_team2.loc[:, 'expwin'] - 2* df_team1.loc[:, 'expwin'] * df_team2.loc[:, 'expwin']))


data = df_team1 - df_team2


df2 = pd.read_csv('NCAA_Tourney_2017.csv').iloc[:, :-1]


df2_team1 = df2.iloc[:, 2:39]
df2_team1.columns = columnName
df2_team2 = df2.iloc[:, 39:76]
df2_team2.columns = columnName

df2_team1.loc[:, 'expwin'] = df2.loc[:, 'team1_adjoe'] ** 11.5 / (df2.loc[:, 'team1_adjde'] ** 11.5
                                                            + df2.loc[:, 'team1_adjoe'] ** 11.5)


df2_team2.loc[:, 'expwin'] = df2.loc[:, 'team2_adjoe'] ** 11.5 / (df2.loc[:, 'team2_adjde'] ** 11.5
                                                            + df2.loc[:, 'team2_adjoe'] ** 11.5)


df2_team1.loc[:, 'log5'] = (df2_team1.loc[:, 'expwin'] - df2_team1.loc[:, 'expwin'] * df2_team2.loc[:, 'expwin']) / \
            ((df2_team1.loc[:, 'expwin'] + df2_team2.loc[:, 'expwin'] - 2* df2_team1.loc[:, 'expwin'] * df2_team2.loc[:, 'expwin']))

df2_team2.loc[:, 'log5'] = (df2_team2.loc[:, 'expwin'] - df2_team1.loc[:, 'expwin'] * df2_team2.loc[:, 'expwin']) / \
            ((df2_team1.loc[:, 'expwin'] + df2_team2.loc[:, 'expwin'] - 2* df2_team1.loc[:, 'expwin'] * df2_team2.loc[:, 'expwin']))

df2_team1.iloc[:, -1]
test_data = df2_team1 - df2_team2
X_train = data
y_train = results.loc[:, 'result']

paramsGB = {'n_estimators': 10, 'max_depth': 5, 'min_samples_split': 2, 'loss': 'deviance'}

original_params = {'n_estimators': 1000, 'max_leaf_nodes': 4, 'max_depth': 5, 'random_state': 2,
                   'min_samples_split': 5}

clf = ensemble.GradientBoostingClassifier(**original_params)

clf.fit(X_train, y_train.values.ravel())

sns.set(font_scale=1, rc={"lines.linewidth": 1.2})

fig = plt.figure(figsize=(20, 10), dpi=80)
fig.suptitle('Predictor Importance', fontsize=20)

plt.subplot(1,1,1)

f = clf.feature_importances_
%matplotlib
f = 100.0 * (f / f.max())

from sklearn import linear_model
from sklearn.calibration import CalibratedClassifierCV
from sklearn.pipeline import make_pipeline

from sklearn.decomposition import PCA

columns = list(X_train.columns[np.argsort(f)[-30:]])
newFeatures = data.loc[:, columns]
X_new = newFeatures
y_new = results.loc[:, 'result']


pca = PCA(10)
pca.fit(X_new)

Using matplotlib backend: MacOSX


PCA(copy=True, iterated_power='auto', n_components=10, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [27]:
import matplotlib.pylab as pylab
from mpl_toolkits.mplot3d import Axes3D 
from matplotlib.patches import Circle


newData = pca.transform(X_new)
fig = pylab.figure(figsize=(10, 7))
ax = Axes3D(fig)

    
ax.scatter(newData[:, 0], newData[:, 1], newData[:, 2], c=y_new, cmap='bwr')

ax.set_xlabel('PC 1')
ax.set_ylabel('PC 2')
ax.set_zlabel('PC 3')

ax.set_title('PCA 3D plot fot Training Data', fontsize=20)

pylab.show()

In [21]:
df = pd.read_csv('pilarz.csv')
df.plot(kind='hist')